# Import các thư viện cần thiết

In [1]:
import requests
from datetime import datetime, timedelta, timezone
import pandas as pd

# Cấu hình chung

In [2]:
TIMEZONE = "Asia/Ho_Chi_Minh"
CSV_OUTPUT = "hcm_weather_overview.csv"
API_URL = "https://archive-api.open-meteo.com/v1/archive"

# Tọa độ lấy từ https://www.geonames.org/1566083/ho-chi-minh-city.html
# Danh sách các tỉnh thành cần lấy
PROVINCES = {
    "Ho_Chi_Minh": (10.82302, 106.62965)
}

# I. Thu thập dữ liệu

## 1. Thiết lập khoảng thời gian

In [3]:
# --- Thời gian lấy dữ liệu ---
# --- Ngày hiện tại (UTC) --- (Có thể cố định 1 ngày để dữ liệu không bị thay đổi)
today = datetime.now(timezone.utc).date()

# --- 2 năm trước ---
start_date = today - timedelta(days=2*365)
end_date = today

print("Start date:", start_date)
print("End date:", end_date)

Start date: 2023-11-25
End date: 2025-11-24


## 2. Các biến thời tiết cần lấy

In [4]:
HOURLY_VARS = [
    "temperature_2m",
    "dew_point_2m",
    "apparent_temperature",
    "relative_humidity_2m",
    "precipitation", # Tổng lượng mưa + tuyết trong giờ (mm)
    "rain", # Lượng mưa (mm) – có thể bỏ ví thành phố HCM không có tuyết
    "wind_speed_10m", 
    "wind_gusts_10m", # Giật gió 10m (km/h) – bỏ nếu chỉ quan tâm gió trung bình
    "wind_direction_10m", # Hướng gió 10m (°) – giữ nếu muốn dự báo hướng gió, bỏ nếu không
    "surface_pressure",
    "pressure_msl",
    "cloud_cover", # Độ che phủ mây tổng (%)
    "cloud_cover_low", # Độ che phủ mây thấp (%) – bỏ nếu chỉ cần tổng quan
    "cloud_cover_mid", # Độ che phủ mây trung bình (%) – bỏ nếu chỉ cần tổng quan
    "cloud_cover_high", # Độ che phủ mây cao (%) – bỏ nếu chỉ cần tổng quan
    "shortwave_radiation",
    "weather_code",
    "vapour_pressure_deficit",
]

## 3. Lấy dữ liệu từ Open-Meteo

In [5]:
def fetch_weather(lat, lon, start_date, end_date, hourly_vars):
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "hourly": ",".join(hourly_vars),
        "timezone": TIMEZONE, # Dữ liệu trả về theo giờ địa phương
        "temperature_unit": "celsius",
        "wind_speed_unit": "kmh",
        "precipitation_unit": "mm",
        "timeformat": "iso8601"
    }
    response = requests.get(API_URL, params=params)
    response.raise_for_status()
    return response.json().get("hourly", {})

all_data = []

for province, (lat, lon) in PROVINCES.items():
    print(f"Lấy dữ liệu cho {province}...")
    hourly = fetch_weather(lat, lon, start_date, end_date, HOURLY_VARS)
    df = pd.DataFrame(hourly)
    if len(PROVINCES) > 1:
        # Thêm thông tin tỉnh và tọa độ
        df["province"] = province
        df["lat"] = lat
        df["lon"] = lon
    all_data.append(df)

full_df = pd.concat(all_data, ignore_index=True)

Lấy dữ liệu cho Ho_Chi_Minh...


## 4. Kiểm tra dữ liệu

In [6]:
full_df.head()

,time,temperature_2m,dew_point_2m,apparent_temperature,relative_humidity_2m,precipitation,rain,wind_speed_10m,wind_gusts_10m,wind_direction_10m,surface_pressure,pressure_msl,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,shortwave_radiation,weather_code,vapour_pressure_deficit
0,2023-11-25T00:00,23.8,21.0,26.9,85,0.0,0.0,8.4,12.6,353,1010.0,1010.7,100,0,100,99,0.0,3,0.45
1,2023-11-25T01:00,23.5,20.9,26.4,85,0.0,0.0,9.4,13.3,353,1009.3,1010.0,100,0,100,99,0.0,3,0.43
2,2023-11-25T02:00,23.4,20.8,26.4,85,0.0,0.0,8.3,13.3,360,1008.7,1009.4,100,2,100,99,0.0,3,0.42
3,2023-11-25T03:00,23.3,20.7,26.6,86,0.0,0.0,6.1,11.9,3,1008.2,1008.9,84,3,11,82,0.0,3,0.41
4,2023-11-25T04:00,22.8,20.5,26.0,87,0.0,0.0,6.5,8.6,357,1008.1,1008.8,97,2,27,95,0.0,3,0.36


In [9]:
full_df.shape

(17544, 19)

In [7]:
full_df.dtypes

time                        object
temperature_2m             float64
dew_point_2m               float64
apparent_temperature       float64
relative_humidity_2m         int64
precipitation              float64
rain                       float64
wind_speed_10m             float64
wind_gusts_10m             float64
wind_direction_10m           int64
surface_pressure           float64
pressure_msl               float64
cloud_cover                  int64
cloud_cover_low              int64
cloud_cover_mid              int64
cloud_cover_high             int64
shortwave_radiation        float64
weather_code                 int64
vapour_pressure_deficit    float64
dtype: object

## 5. Lưu CSV

In [8]:
full_df.to_csv(CSV_OUTPUT, index=False)
print(f"Lưu CSV: {CSV_OUTPUT}")

Lưu CSV: hcm_weather_overview.csv
